In [ ]:
import os
import openai
import csv
openai.api_key = "" #Enter your API key here
openai.Engine.list()

In [ ]:
openai.Answer.create(
  search_model="ada", 
  model="curie",
  question="which puppy is happy?", 
  documents=["Puppy A is happy.", "Puppy B is sad."],
  examples_context="In 2017, U.S. life expectancy was 78.6 years.", 
  examples=[["What is human life expectancy in the United States?","78 years."]],
  max_tokens=5,
  stop=["\n", "<|endoftext|>"],
)

In [ ]:
import json


max_tokens = 2

total_samples = 0
correct_predictions = 0

train_address = 'Data\\PubMed-QA-train-10.json'
test_address = 'Data\\PubMed-QA-test.json'

test_file = open(test_address)
test_data = json.load(test_file)

for pubmed_id in test_data:
    total_samples += 1
    print(pubmed_id)
    
    sample = test_data[pubmed_id]
    
    question = sample['QUESTION']
    print('Question:', question)
    
    context = sample['CONTEXTS']
    print('Context:', context)
    
    answer = sample['final_decision']
    print('Answer:', answer)
    #------------------------------------------
    train_file = open(train_address)
    train_data = json.load(train_file)
    
    train_context = ""
    train_samples = []
    
    for train_id in train_data:
        train_sample = train_data[train_id]
        train_question = train_sample['QUESTION']
        train_answer = train_sample['final_decision']
        train_pair = []
        train_pair.append(train_question)
        train_pair.append(train_answer)
        
        train_samples.append(train_pair)
        
        train_contexts = train_sample['CONTEXTS']
        
        for c in train_contexts:
            train_context += c + ' '
    #------------------------------------------
    print('Train context:', train_context)
    print('Train samples:', train_samples)
    response = openai.Answer.create(search_model="curie", model="curie", question=question, documents=context, examples_context=train_context, examples=train_samples, max_tokens=max_tokens, stop=["\n", "<|endoftext|>"])
    print('\n**********\n')    
    print(response)
    answers = response['answers']
    GPT3_answer = answers[0]
    print('Test answer:', answer, '----- GPT3 answer:', GPT3_answer)
    if(GPT3_answer == answer):
        correct_predictions += 1
    
    print('----------------------------------------------------------------------------------')
    
accuracy = correct_predictions / total_samples
print('Total samples:', total_samples)
print('Correct predictions:', correct_predictions)
print('Accuracy:', accuracy)